In [2]:
import pandas as pd
import numpy as np

# BIKES IN EDINBURGH
1. [Zadání](#1)
2. [Úprava dat](#2)
    1. [Jednotlivé datasety](#2.1)
        1. [Bikes](#2.1.1)
        2. [Weather](#2.1.2)
        3. [Stations](#2.1.3)
    2. [Sjednocený dataset](#2.2)
3. [Analýza](#3)



## Zadání <a name="1"></a>



V Edinburghu, stejně jako v dalších městech, funguje systém "bike sharing" - ve městě jsou stanice s koly, člověk si může nějaké půjčit a potom ho vrátit v nějaké další stanici. Problém je, že v některých stanicích se kola pravidelně hromadí a jinde naopak chybí. Provozovatel kol, firma Just Eat Cycles, zadala projekt, jehož cílem je systém zefektivnit.

Coby datový analytik jste součástí týmu, který na projektu pracuje. Vaším úkolem je zpracovat relevantní data a zjistit z nich informace užitečné pro zbytek týmu. Máte k dispozici data o všech výpůjčkách (na ENGETO databázi v tabulce edinburgh_bikes). Proveďte standardní deskriptivní statistiku dat. Také zjistěte minimálně následující informace:

identifikujte aktivní a neaktivní stanice
identifikujte nejfrekventovanější stanice
identifikujte stanice, na kterých se kola hromadí a stanice, kde potenciálně chybí
spočítejte vzdálenosti mezi jednotlivými stanicemi
jak dlouho trvá jedna výpůjčka? Najděte odlehlé hodnoty, zobrazte histogram
Analýza poptávky:

zobrazte vývoj poptávky po půjčování kol v čase
identifikujte příčiny výkyvů poptávky
zjistěte vliv počasí na poptávku po kolech (údaje o počasí v Edinburghu jsou v tabulce edinburgh_weather)
půjčují si lidé kola více o víkendu než během pracovního týdne?
Výstupem analýzy bude ideálně jupyter notebook nebo colab notebook uložený na Vašem githubu. Výstupy Vaší analýzy musí být zřejmé i bez spouštění kódu. V textových buňkách popište, jak jste postupovali a napište své závěry. Pokud v zadání chybí nějaká informace, doplňte ji podle vlastního uvážení, své rozhodnutí uveďte a vysvětlete v notebooku.

## Úprava dat <a name="2"></a>

V první řadě je potřeba zkontrolovat dostupné datasety a provést základní analýzu a úpravu datových formátu pro lepší práci s daty. Databáze jsem si načetl do jednotlivých Pandas dataframů pomocí `data = pd.read_sql(sql=query, con=engine)` a uložil jako formát pickle `data.to_pickle("data_basic.pkl")`, s kterým budu následně pracovat.

### Jednotlivé datasety <a name="2.1"></a>

#### Bikes <a name="2.1.1"></a>

In [3]:
dfb = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/bikes_first.pkl")

In [4]:
dfb.head()

,index,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
0,0,2018-09-15 08:52:05,2018-09-15 09:11:48,1182,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,St Andrew Square,North East corner,55.954728,-3.192653
1,1,2018-09-15 09:24:33,2018-09-15 09:41:09,995,259,St Andrew Square,North East corner,55.954749,-3.192774,262,Canonmills,near Tesco's,55.962804,-3.196284
2,2,2018-09-15 09:48:54,2018-09-15 10:46:40,3466,262,Canonmills,near Tesco's,55.962804,-3.196284,250,Victoria Quay,Entrance to Scottish Government Office,55.977638,-3.174116
3,3,2018-09-16 12:01:36,2018-09-16 12:25:26,1430,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,254,Kings Building 3,Kings Building House,55.923479,-3.175385
4,4,2018-09-16 12:03:43,2018-09-16 12:11:16,452,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,253,Kings Building 2,Sanderson Building,55.923202,-3.171646


In [5]:
dfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438259 entries, 0 to 438258
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   index                      438259 non-null  int64  
 1   started_at                 438259 non-null  object 
 2   ended_at                   438259 non-null  object 
 3   duration                   438259 non-null  int64  
 4   start_station_id           438259 non-null  int64  
 5   start_station_name         438259 non-null  object 
 6   start_station_description  435549 non-null  object 
 7   start_station_latitude     438259 non-null  float64
 8   start_station_longitude    438259 non-null  float64
 9   end_station_id             438259 non-null  int64  
 10  end_station_name           438259 non-null  object 
 11  end_station_description    435256 non-null  object 
 12  end_station_latitude       438259 non-null  float64
 13  end_station_longitude      43

V první řadě je vhodné upravit datový typ object u sloupců ***started_at*** a ***ended_at*** na formát datetime64 pro lepší práci s datem a časem.

In [6]:
dfb['started_at'] = pd.to_datetime(dfb['started_at'], format="%Y-%m-%d %H:%M" )
dfb['ended_at'] = pd.to_datetime(dfb['ended_at'], format="%Y-%m-%d %H:%M" )

Pro lepší přehlednost přejmenuji sloupce na kratší názvy.

In [7]:
dfb = dfb.rename(columns={"started_at": "start", "ended_at": "end",
        "start_station_id": "start_id", "start_station_name": "start_name",
        "start_station_description": "start_description",
        "start_station_latitude": "start_latitude",
        "start_station_longitude": "start_longitude","end_station_id": "end_id",
        "end_station_name": "end_name", 
        "end_station_description": "end_description",
        "end_station_latitude": "end_latitude",
        "end_station_longitude": "end_longitude"
        })

In [8]:
dfb = dfb.drop(columns= "index")

In [9]:
dfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438259 entries, 0 to 438258
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   start              438259 non-null  datetime64[ns]
 1   end                438259 non-null  datetime64[ns]
 2   duration           438259 non-null  int64         
 3   start_id           438259 non-null  int64         
 4   start_name         438259 non-null  object        
 5   start_description  435549 non-null  object        
 6   start_latitude     438259 non-null  float64       
 7   start_longitude    438259 non-null  float64       
 8   end_id             438259 non-null  int64         
 9   end_name           438259 non-null  object        
 10  end_description    435256 non-null  object        
 11  end_latitude       438259 non-null  float64       
 12  end_longitude      438259 non-null  float64       
dtypes: datetime64[ns](2), float64(4), int64(3), 

In [11]:
dfb["start"].min()

Timestamp('2018-09-15 08:52:05')

In [12]:
dfb["start"].max()

Timestamp('2021-06-30 23:58:33')

Dataframe obsahuje **438 259** záznamů a **14** sloupců s informacemi od data **2018-09-15** do **2021-06-30**.

Upravený dataframe si uložím to_pickle a dále budu pracovat už jenom s upraveným picklem.

In [13]:
dfb.to_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/bikes.pkl")

#### Weather <a name="2.1.2"></a>

In [14]:
# Načtení pickle do Dataframu.
dfw = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/weather_first.pkl")

In [15]:
dfw.head()

,time,temp,feels,wind,gust,rain,humidity,cloud,pressure,vis,date
0,00:00,11 °c,11 °c,9 km/h from S,19 km/h,0.0 mm,79%,13%,1020 mb,Excellent,2018-09-01
1,03:00,13 °c,12 °c,11 km/h from SSW,19 km/h,0.0 mm,76%,96%,1020 mb,Excellent,2018-09-01
2,06:00,14 °c,13 °c,11 km/h from SSW,19 km/h,0.0 mm,84%,100%,1020 mb,Excellent,2018-09-01
3,09:00,14 °c,13 °c,14 km/h from SSW,23 km/h,0.1 mm,88%,78%,1021 mb,Excellent,2018-09-01
4,12:00,16 °c,16 °c,15 km/h from SSW,22 km/h,0.0 mm,87%,87%,1021 mb,Excellent,2018-09-01


In [16]:
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6336 entries, 0 to 6335
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   time      6336 non-null   object
 1   temp      6336 non-null   object
 2   feels     6336 non-null   object
 3   wind      6336 non-null   object
 4   gust      6336 non-null   object
 5   rain      6336 non-null   object
 6   humidity  6336 non-null   object
 7   cloud     6336 non-null   object
 8   pressure  6336 non-null   object
 9   vis       6336 non-null   object
 10  date      6336 non-null   object
dtypes: object(11)
memory usage: 544.6+ KB


In [17]:
# sjednocení datumu s časem a převedení na datetime64
dfw['time'] = pd.to_datetime((dfw['date'] + ' ' + dfw['time']), format="%Y-%m-%d %H:%M" )

Odstraním sloupce, které nebudu používat a přejmenuji názvy sloupců a nastavím datové formáty jednotlivých sloupců.

In [18]:
# vyjmutí jednotek ze sloupců a nastavení datového formátu
dfw["temp"] = dfw["temp"].str.extract('(\d+)').astype(int)
dfw['wind'] = dfw['wind'].str.extract('(\d+)').astype(int)
dfw["gust"] = dfw["gust"].str.extract('(\d+)').astype(int)
dfw["rain"] = dfw["rain"].str.extract('(\d+)').astype(float)

In [19]:
# odebrání nepotřebných sloupců
dfw = dfw.drop(columns=["feels","humidity","cloud","pressure","vis", "date"])

In [20]:
# přejmenování sloupců na název a jednotku
dfw = dfw.rename(columns={"time":"datetime", "temp":"temp_c",
                          "wind":"wind_kmh", "gust":"gust_kmh",
                          "rain":"rain_mm"})

In [21]:
dfw.head()

,datetime,temp_c,wind_kmh,gust_kmh,rain_mm
0,2018-09-01 00:00:00,11,9,19,0.0
1,2018-09-01 03:00:00,13,11,19,0.0
2,2018-09-01 06:00:00,14,11,19,0.0
3,2018-09-01 09:00:00,14,14,23,0.0
4,2018-09-01 12:00:00,16,15,22,0.0


In [22]:
dfw['datetime'].min()

Timestamp('2018-09-01 00:00:00')

In [23]:
dfw['datetime'].max()

Timestamp('2020-10-31 21:00:00')

In [24]:
# uložení jako pickle pro pozdější práci
dfw.to_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/weather.pkl")

Upravený dataframe obsahuje **6336** záznamů a 5 sloupců s informacemi od data **2018-09-01 00:00** do **2020-10-31 21:00**.

#### Stations <a name="**2.1.3**"></a>

* Z upraveného dataframu **bikes.pkl** si vytvořím dataframe pouze s unikátními stanicemi bez časových záznamů. Tento dataframe budu využívat jako seznam všech stanic s informacemi o ních.  
* K datasetu přidám sloupec s početem výpujček a vrácení kol v jednotlivých stanicích.
* Stanice, které jsou k dispozici zjistíme ze **start_id** a **end_id**

In [25]:
dfs = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/bikes.pkl")

In [26]:
dfs_start = dfs.drop(columns=["start","end","duration","end_id","end_name","end_description","end_latitude","end_longitude"])

In [27]:
dfs_start = dfs_start.drop_duplicates('start_id').reset_index(drop=True)

In [28]:
dfs_start = dfs_start.rename(
            columns={"start_name": "name",
                     "start_id": "id",
                 "start_description": "description",
                 "start_latitude": "lat",
                 "start_longitude": "lon"})

In [29]:
dfs_end = dfs.drop(columns=["start","end","duration","start_id","start_name","start_description","start_latitude","start_longitude"])

In [30]:
dfs_end = dfs_end.drop_duplicates('end_id').reset_index(drop=True)

In [31]:
dfs_end = dfs_end.rename(
            columns={"end_name": "name",
                     "end_id": "id",
                 "end_description": "description",
                 "end_latitude": "lat",
                 "end_longitude": "lon"})

In [32]:
dfs_new = dfs_start.append(dfs_end, ignore_index = True)

In [33]:
dfs_new = dfs_new.drop_duplicates('id').reset_index(drop=True)

Nový dataframe, který obsahuje všechny bikes stanice. Dataset obsahuje **200 stanic**.

In [34]:
dfs_new["id"].count()

200

Dále přidáme ke každé stanici sloupec s početem vypůjčených kol a vrácených kol.

In [35]:
start_frequency = dfs["start_id"].value_counts().rename_axis("id").reset_index(name = "start_frequency")

In [36]:
end_frequency = dfs["end_id"].value_counts().rename_axis("id").reset_index(name = "end_frequency")

In [37]:
dfs_final = dfs_new.merge(start_frequency, how="outer", on = "id")

In [38]:
dfs_final = dfs_final.merge(end_frequency, how= "outer", on= "id")

In [39]:
dfs_final.head()

,id,name,description,lat,lon,start_frequency,end_frequency
0,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,8134.0,4980.0
1,259,St Andrew Square,North East corner,55.954749,-3.192774,12218.0,8372.0
2,262,Canonmills,near Tesco's,55.962804,-3.196284,9551.0,11940.0
3,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,254.0,196.0
4,253,Kings Building 2,Sanderson Building,55.923202,-3.171646,4096.0,4557.0


Z dostupných stanic zjistíme, v kterých nebylo nikdy zapůjčeno kolo a nebylo nikdy vráceno kolo.

In [40]:
dfs_final[dfs_final["start_frequency"].isna()]

,id,name,description,lat,lon,start_frequency,end_frequency
198,242,Virtual Depot,Virtual Depot,55.972402,-3.155794,NaN,2.0
199,280,Smarter Travel Station,The Street,53.395525,-2.990138,NaN,3.0


In [41]:
dfs_final[dfs_final["end_frequency"].isna()]

,id,name,description,lat,lon,start_frequency,end_frequency
182,1857,City Chambers Launch Station,Temporary station at City Chambers,55.950222,-3.19027,1.0,NaN


In [42]:
dfs_final.to_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/stations.pkl")

V Edinburghu se nachází **200** stanic.  
Připravený dataframe uložím do **stations.pkl**, s kterým budu následně pracovat.

#### Sjednocený dataset <a name="2.2"></a>

Sjednocený dataset záznamů o vypůjčení kol s informacemi o počasí podle data.

In [43]:
# Načtení dat
dfb1 = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/bikes.pkl")
dfw2 = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/weather.pkl")

In [44]:
# sjednotím data o výpujčkách kol s daty o počasí. Dataframe bikes omezím pouze na dostupné údaje o počasí.
last = str(dfw2.datetime.max())
merged_df = pd.merge_asof(dfb1[dfb1["start"] <= last], dfw2,
                                 left_on="start", right_on="datetime", 
                                 direction='nearest',
                                 allow_exact_matches=False).drop(columns="datetime")

In [45]:
merged_df.head()

,start,end,duration,start_id,start_name,start_description,start_latitude,start_longitude,end_id,end_name,end_description,end_latitude,end_longitude,temp_c,wind_kmh,gust_kmh,rain_mm
0,2018-09-15 08:52:05,2018-09-15 09:11:48,1182,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,St Andrew Square,North East corner,55.954728,-3.192653,10,16,23,0.0
1,2018-09-15 09:24:33,2018-09-15 09:41:09,995,259,St Andrew Square,North East corner,55.954749,-3.192774,262,Canonmills,near Tesco's,55.962804,-3.196284,10,16,23,0.0
2,2018-09-15 09:48:54,2018-09-15 10:46:40,3466,262,Canonmills,near Tesco's,55.962804,-3.196284,250,Victoria Quay,Entrance to Scottish Government Office,55.977638,-3.174116,10,16,23,0.0
3,2018-09-16 12:01:36,2018-09-16 12:25:26,1430,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,254,Kings Building 3,Kings Building House,55.923479,-3.175385,14,32,47,0.0
4,2018-09-16 12:03:43,2018-09-16 12:11:16,452,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,253,Kings Building 2,Sanderson Building,55.923202,-3.171646,14,32,47,0.0


In [46]:
# uložení jako pickle
merged_df.to_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/merged.pkl")

## Analýza<a name="3"></a>

### Vzdálenosti stanic<a name="3.1"></a>

In [47]:
from sklearn.neighbors import DistanceMetric

In [48]:
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/bikes/stations.pkl")

In [49]:
df = df.sort_values(by="id")

https://medium.com/@danalindquist/finding-the-distance-between-two-lists-of-geographic-coordinates-9ace7e43bb2f

In [50]:
# převedení na radiány
df['lat'] = np.radians(df['lat'])
df['lon'] = np.radians(df['lon'])

In [51]:
# vytvoření matice pomocí knihovny
dist = DistanceMetric.get_metric('haversine')

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_distance_metric.py:14: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  category=FutureWarning,


In [52]:
# radius země v km
earth_radius = 6373
distance = pd.DataFrame(dist.pairwise((df[['lat','lon']]) * earth_radius).round(2),  columns=df.id, index=df.id)

In [53]:
distance

id,171,183,189,225,241,242,246,247,248,249,...,1868,1869,1870,1871,1874,1877,2259,2263,2265,2268
id,,,,,,,,,,,,,,,,,,,,,
171,0.00,0.96,0.80,1.03,1.84,1.84,0.64,1.92,0.33,1.15,...,1.07,0.89,0.23,1.09,1.30,0.70,2.47,0.58,1.05,1.51
183,0.96,0.00,0.19,0.81,1.13,1.13,1.59,1.75,0.66,1.78,...,1.58,1.60,1.18,0.57,1.36,1.59,1.51,1.37,1.95,0.80
189,0.80,0.19,0.00,0.70,1.18,1.18,1.44,1.88,0.48,1.74,...,1.56,1.53,1.02,0.69,1.43,1.46,1.67,1.18,1.76,0.83
225,1.03,0.81,0.70,0.00,0.83,0.83,1.51,2.54,0.79,2.18,...,2.09,1.92,1.22,1.37,2.11,1.69,1.65,0.97,1.47,0.58
241,1.84,1.13,1.18,0.83,0.00,0.00,2.33,2.10,1.54,2.91,...,2.69,2.70,2.04,1.54,2.28,2.52,0.85,1.76,2.08,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,0.70,1.59,1.46,1.69,2.52,2.52,0.26,1.53,1.02,0.53,...,0.59,0.32,0.49,1.42,1.05,0.00,2.81,0.82,0.78,2.21
2259,2.47,1.51,1.67,1.65,0.85,0.85,3.07,1.30,2.16,2.32,...,2.23,2.50,2.68,1.51,1.77,2.81,0.00,2.60,2.65,1.08
2263,0.58,1.37,1.18,0.97,1.76,1.75,0.58,2.34,0.75,1.35,...,1.39,1.13,0.54,1.66,1.78,0.82,2.60,0.00,0.59,1.55


In [54]:
distance.max().sort_values(ascending=False)

id
1723    3.13
1767    3.13
225     3.13
1819    3.13
290     3.13
        ... 
1731    2.79
2268    2.78
965     2.77
1057    2.73
1027    2.71
Length: 200, dtype: float64